In [51]:
from imageai.Detection import ObjectDetection
import os
from imageai.Detection.Custom import DetectionModelTrainer
import xml.etree.ElementTree as ET 
from imageai.Detection.Custom import CustomObjectDetection
import pandas as pd
import pytesseract
import cv2 as cv
from PIL import Image
import smtplib 
import re

# Misc Functions

In [52]:
def sendMail(msg):
    sender="alexkay72850@gmail.com"
    sender_pwd="alexkay728"
    reciever="abhishekk728@yahoo.com"
    
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls()
    s.login(sender, sender_pwd)

    # smtp.mail.yahoo.com
    # smtp.gmail.com
    
    message = 'Subject: Item Alert\n\n{}'.format(msg)
    
    s.sendmail(sender, reciever, message) 
    s.quit()
    
def extractImage(name,points):
    image = cv.imread(name)
    clone = image.copy()
    crop_img=clone[points[1]:points[3],points[0]:points[2]]
    
    return crop_img

def extractText(image):
    
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    gray = cv.threshold(gray, 0, 255,cv.THRESH_BINARY | cv.THRESH_OTSU)[1]
    
    #gray=invert(gray)

    #gray = cv.medianBlur(gray, 3)
    
    #cv.imwrite('ocr.png', gray)
    
    text = pytesseract.image_to_string(gray,lang='eng')
    return text,gray

def extractObjectText(top_n,name_lst,filename,extract):
    points=top_n[name_lst.index(extract)]['box_points']
    cropped_image = extractImage(filename,points)
    label_text,gray = extractText(cropped_image)
    
    cv.imshow(extract,gray)
    
    
    return label_text

# Resizing Images

In [ ]:
for root, dirs, files in os.walk("images"):
    for i,f in enumerate(files):
        img = cv.imread(os.path.join(root,f))

        #print('Original Dimensions : ',img.shape)

        scale_percent = 20 # percent of original size
        width = int(img.shape[1] * scale_percent / 100)
        height = int(img.shape[0] * scale_percent / 100)
        dim = (width, height)
        # resize image
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

        #print('Resized Dimensions : ',resized.shape)
        cv.imwrite('train/'+str(i)+'.jpg', resized, [int(cv.IMWRITE_JPEG_QUALITY), 80])
        print(i)

# Generating Classes to Predict 

In [38]:
classes = set()
for root, dirs, files in os.walk("dataset/train"):
    for f in files:
        part=f.split('.')
        if(part[1]=='xml'):
            tree=ET.parse(os.path.join(root,f))
            xml_root = tree.getroot()
            for item in xml_root.findall('./object/name'):
                classes.add(item.text)

# Model Training

In [ ]:
trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="dataset")
trainer.setTrainConfig(object_names_array=classes, batch_size=4, num_experiments=10,train_from_pretrained_model="yolov3_model.h5")
trainer.trainModel()

# To Test the Model Start from here

## Note: First Execute the Libraries Block and Misc Function Block before executing below blocks 

# Loading Trained Model for Object Detection

In [3]:
detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("dataset/models/detection_model-ex-010--loss-0012.245.h5") 
detector.setJsonPath("dataset/json/detection_config.json")
detector.loadModel()

Instructions for updating:
Colocations handled automatically by placer.


# Provide Filename to Detect Objects

In [4]:
filename = "26.jpg" #filename of image

detections,ext = detector.detectObjectsFromImage(input_image=filename, output_image_path="detected.jpg",extract_detected_objects=True)

# Sorting Detected Objects With Higher Probability

In [5]:
detections=sorted(detections,key=lambda v: v['percentage_probability'],reverse=True)
detections

[{'name': 'Time',
  'percentage_probability': 99.56734776496887,
  'box_points': [123, 0, 595, 47]},
 {'name': 'Label',
  'percentage_probability': 99.48127865791321,
  'box_points': [166, 518, 414, 689]},
 {'name': 'Location',
  'percentage_probability': 98.84781241416931,
  'box_points': [332, 64, 612, 207]},
 {'name': 'Chune Fried rice',
  'percentage_probability': 98.40279817581177,
  'box_points': [38, 104, 563, 441]},
 {'name': 'Label',
  'percentage_probability': 97.08787798881531,
  'box_points': [332, 64, 612, 207]},
 {'name': 'Pork Skin',
  'percentage_probability': 96.31786942481995,
  'box_points': [38, 104, 563, 441]},
 {'name': 'Label',
  'percentage_probability': 93.37312579154968,
  'box_points': [38, 104, 563, 441]},
 {'name': 'Golden Saba',
  'percentage_probability': 89.7739827632904,
  'box_points': [166, 518, 414, 689]},
 {'name': 'Location',
  'percentage_probability': 84.96984839439392,
  'box_points': [166, 518, 414, 689]},
 {'name': 'Angel Soft',
  'percentage_

# Extracting Top N Detected objects

In [6]:
name_lst=[]
top_n=[]

limit=5  #change the value to vary number of top N objects

for obj in detections:
    if obj['name'] not in name_lst:
        top_n.append(obj)
        name_lst.append(obj['name'])
        
print(top_n[0:limit],name_lst[0:limit])

[{'name': 'Time', 'percentage_probability': 99.56734776496887, 'box_points': [123, 0, 595, 47]}, {'name': 'Label', 'percentage_probability': 99.48127865791321, 'box_points': [166, 518, 414, 689]}, {'name': 'Location', 'percentage_probability': 98.84781241416931, 'box_points': [332, 64, 612, 207]}, {'name': 'Chune Fried rice', 'percentage_probability': 98.40279817581177, 'box_points': [38, 104, 563, 441]}, {'name': 'Pork Skin', 'percentage_probability': 96.31786942481995, 'box_points': [38, 104, 563, 441]}] ['Time', 'Label', 'Location', 'Chune Fried rice', 'Pork Skin']


# Performing OCR and Generating Message to Email

In [53]:
label=extractObjectText(top_n,name_lst,filename,'Label')
location=extractObjectText(top_n,name_lst,filename,'Location')
time=extractObjectText(top_n,name_lst,filename,'Time')

#print("Label ",label)
#print("Location ",location)
#print("Time ",time)

cv.waitKey(0)
cv.destroyAllWindows()

if(str(location)!='None'):
    location_str = ' '.join(location.split('\n'))
else:
    location_str='Southeast America'

if(time!=''):
    time = re.search(r'\d{1}:\d{2}:\d{2}', time)
    if(str(time)=='None'):
        time = re.search(r'\d{1}:\d{1,2}', time)
        if(str(time)=='None'):
            time = '00:00:00'
        else:
            time = time.group()
    else:
        time = time.group()
else:
    time='00:00:00'

if(label!=''):
    price = re.search(r'(\d{1,})([. ])(\d{1,})', label)
    
    if(str(price)!='None'):
        price = price.group()
    else:
        price='0'

    label_str = ' '.join(label.split(price))
else:
    price='0'
    label_str='x'

message=""

if 'Void' in name_lst[0:limit]:
    print('Shelf Has an Empty Area')
        
    message="Item name "+label_str+" of price "+price+"is missing from shelf Taken on "+time+" At "+location_str
    
else:
    item = name_lst[3]
    
    count=1
    
    for name in name_lst:
        if(name==item):
            count+=1
    
    print("Item Count : ",count)
    
    if(label_str!=item):
        message="Item "+item+" does not match label "+label_str
    else:
        message="Only "+count+" of item "+item+" are left"
        
    
sendMail(message.encode('UTF-8'))

Item Count :  2
